# 时间序列预测模型

## 📋 概述

时间序列预测是股票分析中的重要方法，通过分析历史价格数据来预测未来价格走势。本章节将详细介绍时间序列预测模型在股票分析中的应用，包括ARIMA、SARIMA、Prophet等模型。

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过实际代码示例理解时间序列预测模型。


In [ ]:
# 安装必要的库（如果还没有安装）
# !pip install pandas numpy matplotlib statsmodels prophet

# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")


## 📖 核心内容

### 1. 时间序列预测的基本概念

#### 1.1 什么是时间序列

**时间序列（Time Series）**：
- 按时间顺序排列的数据序列
- 股票价格是典型的时间序列
- 具有趋势、季节性、周期性等特征

#### 1.2 时间序列特征

**趋势（Trend）**：
- 长期上升或下降趋势
- 反映股票的基本走势

**季节性（Seasonality）**：
- 周期性重复的模式
- 如月度、季度效应

**周期性（Cyclical）**：
- 不规则的周期性波动
- 与经济周期相关

让我们先准备数据并分析时间序列特征：


In [ ]:
# 生成示例时间序列数据
np.random.seed(42)
n_days = 500
dates = pd.date_range(start='2022-01-01', periods=n_days, freq='D')
dates = dates[dates.weekday < 5]  # 只保留工作日

# 生成带趋势和季节性的价格数据
trend = np.linspace(10, 15, len(dates))
seasonal = 0.5 * np.sin(2 * np.pi * np.arange(len(dates)) / 20)  # 20天周期
noise = np.random.randn(len(dates)) * 0.2
prices = trend + seasonal + noise

price_series = pd.Series(prices, index=dates, name='Close')

print(f"时间序列数据准备完成！共 {len(price_series)} 条记录")
print(f"价格范围：{price_series.min():.2f} - {price_series.max():.2f}")

# 可视化时间序列
fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

axes[0].plot(price_series.index, price_series.values, linewidth=1.5, color='black', alpha=0.7)
axes[0].set_title('股票价格时间序列', fontsize=14, fontweight='bold')
axes[0].set_ylabel('价格')
axes[0].grid(True, alpha=0.3)

# 时间序列分解
try:
    decomposition = seasonal_decompose(price_series, model='additive', period=20)
    
    axes[1].plot(decomposition.trend.index, decomposition.trend.values, 
                label='趋势', linewidth=2, color='blue')
    axes[1].plot(decomposition.seasonal.index, decomposition.seasonal.values, 
                label='季节性', linewidth=1.5, color='green', alpha=0.7)
    axes[1].plot(decomposition.resid.index, decomposition.resid.values, 
                label='残差', linewidth=1, color='red', alpha=0.5)
    axes[1].set_title('时间序列分解', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('日期')
    axes[1].set_ylabel('值')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
except Exception as e:
    print(f"时间序列分解失败：{e}")

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# ARIMA模型预测
def arima_forecast(series, order=(1, 1, 1), forecast_steps=30):
    """
    ARIMA模型预测
    
    参数:
    series: 时间序列
    order: (p, d, q) 参数
    forecast_steps: 预测步数
    """
    try:
        model = ARIMA(series, order=order)
        fitted_model = model.fit()
        
        # 预测
        forecast = fitted_model.forecast(steps=forecast_steps)
        forecast_index = pd.date_range(start=series.index[-1] + pd.Timedelta(days=1), 
                                      periods=forecast_steps, freq='D')
        forecast_series = pd.Series(forecast, index=forecast_index)
        
        return fitted_model, forecast_series
    except Exception as e:
        print(f"ARIMA模型拟合失败：{e}")
        return None, None

# 划分训练集和测试集
train_size = int(len(price_series) * 0.8)
train_data = price_series[:train_size]
test_data = price_series[train_size:]

# 训练ARIMA模型
print("训练ARIMA模型...")
arima_model, arima_forecast = arima_forecast(train_data, order=(2, 1, 2), forecast_steps=len(test_data))

if arima_model is not None:
    print("\nARIMA模型摘要：")
    print(arima_model.summary())
    
    # 可视化预测结果
    fig, ax = plt.subplots(figsize=(15, 6))
    
    ax.plot(train_data.index, train_data.values, label='训练数据', linewidth=1.5, color='blue', alpha=0.7)
    ax.plot(test_data.index, test_data.values, label='实际值', linewidth=1.5, color='black', alpha=0.7)
    ax.plot(arima_forecast.index, arima_forecast.values, label='ARIMA预测', 
           linewidth=2, color='red', linestyle='--', alpha=0.7)
    
    ax.set_title('ARIMA模型预测结果', fontsize=14, fontweight='bold')
    ax.set_xlabel('日期')
    ax.set_ylabel('价格')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # 计算预测误差
    mse = mean_squared_error(test_data.values, arima_forecast.values)
    mae = np.mean(np.abs(test_data.values - arima_forecast.values))
    
    print(f"\n预测误差：")
    print(f"MSE: {mse:.4f}")
    print(f"MAE: {mae:.4f}")
else:
    print("ARIMA模型训练失败，请检查数据")


### 3. 其他时间序列模型

#### 3.1 移动平均模型

简单的移动平均可以作为基准模型：

#### 3.2 指数平滑模型

指数平滑适用于有趋势和季节性的数据。

## 💡 关键要点总结

1. **时间序列特征**：趋势、季节性、周期性
2. **ARIMA模型**：自回归积分滑动平均模型
3. **模型选择**：根据数据特征选择合适的模型
4. **模型评估**：使用MSE、MAE等指标评估

## 🛠️ 工具与软件

### 时间序列分析工具

- **statsmodels**：Python时间序列分析库
- **prophet**：Facebook开发的时间序列预测库
- **pandas**：数据处理

### 注意事项

- ✅ **数据平稳性**：ARIMA要求数据平稳
- ✅ **参数选择**：需要选择合适的p、d、q参数
- ✅ **模型验证**：在测试集上验证模型
- ⚠️ **过拟合风险**：避免过度拟合历史数据
- ⚠️ **外推风险**：长期预测准确性会下降

## 🔗 相关知识点

- [机器学习预测模型](./机器学习预测模型.md)
- [深度学习预测模型](./深度学习预测模型.md)
- [策略设计原理](../../05_量化交易/02_策略开发/策略设计原理.ipynb)

## 📚 拓展阅读

- 《时间序列分析》- 时间序列经典教材
- 《Python时间序列分析》- 使用Python进行时间序列分析
- 时间序列预测相关论文 - 最新研究方法
